# Projekt

## Zadanie 1

Przygotuj **`100 dokumentów`** w języku angielskim (nie za długich, po około pół strony) i zapisz je do
jednego pliku tekstowego. Dokumenty oddzielaj przynajmniej jedną pustą linią. Podobnie przygotuj
plik z **`5 zapytaniami`** (query, Q). Wczytaj dokumenty oraz Q do R (możesz użyć funkcji read.docs()
z pliku TM_fun.R). Używając pakietu tm utwórz z wczytanych dokumentów oraz Q korpus i dokonaj
standardowego prepocessingu (zamiana na małe litery, usunięcie liczb, usunięcie znaków
przystankowych, stemming Portera, usunięcie słów z stoplisty). Następnie zbuduj macierz TDM
(w wariantach: **`binarnym`**, **`BOW`** oraz **`TFIDF`**) i dla wszystkich Q znajdź po 5 najbardziej adekwatnych
dokumentów. Na koniec wyświetl podsumowanie danych w postaci chmury słów. Powtórz cały
eksperyment z tą różnicą, że tym razem nie używaj stemmera Portera (ani żadnego innego).
Porównaj i skomentuj oba otrzymane wyniki.

## # BIBLIOTEKI

In [ ]:
# 1. Narzędzia podstawowe
import pandas as pd
import numpy as np
import re

# 2. Wizualizacja
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# 3. Przetwarzanie Języka Naturalnego (NLP) - biblioteka NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# 4. Uczenie Maszynowe i Analiza Tekstu - Biblioteka Scikit-Learn (sklearn)
# CountVectorizer: Zamienia tekst na macierz zliczeń (Bag of Words / Binary)
# TfidfVectorizer: Zamienia tekst na macierz ważoną (statystyka TF-IDF)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# cosine_similarity: Funkcja do obliczania podobieństwa między wektorami (dokumentami a zapytaniem)
from sklearn.metrics.pairwise import cosine_similarity

# do zliczenia wystąpień słów w dokumencie
from collections import Counter


# konfiguracja nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

## # OBIEKTY
Tworzymy obiekty, których będziemy używać w funkcji

In [ ]:
# 1. stopwords - lista słów do usunięcia (np. "the", "a", "in") dla języka angielskiego
stop_words = set(stopwords.words('english'))

# 2. Stemmer Portera - algorytm, który ucina końcówki słów (np. "running" -> "run")
ps = PorterStemmer()

## # PREPROCESSING
#### Czyszczenie tekstu zgodnie z wymaganiami zadania:
- `zamiana na małe litery`
- `usunięcie liczb`
- `usunięcie znaków przystankowych`
- `stemming Portera`
- `usunięcie słów z stoplisty`

In [ ]:
def preprocess_text(text, use_stemming=True):
    """
    argumenty: 
    text - surowy tekst do wyczyszczenia
    use_stemming - czy używać stemming, dla porównania jak działa to bez stemmingu trzeba zamienić 'True' na 'False'
    """
    
    # 1. zamiana na małe litery
    text = text.lower()
    
    # 2. usunięcie liczb i znaków przystankowych
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. tokenizacja 
    words = text.split()
    
    # 4. usunięcie stopwords i stemming
    cleaned_words = [] # tworzymy nową listę
    for word in words:
        if word not in stop_words:
            if use_stemming:
                cleaned_words.append(ps.stem(word))
            else:
                cleaned_words.append(word) 
    return " ".join(cleaned_words)
        

## # DOKUMENTY I ZAPYTANIA

In [ ]:
# 1. wczytujemy dokumenty (docs)
with open('dokumenty.txt', 'r', encoding='utf-8') as f:
    raw_docs = f.read().split('\n\n')

# 2. wczytujemy zapytania (queries)
with open('zapytania.txt', 'r', encoding='utf-8') as f:
    raw_queries = f.read().split('\n')

# 3. przetwarzamy dokumenty (wersja ze stemmingiem (True); wersja bez stemmingu (False))
clean_docs_stemmed = [preprocess_text(doc, use_stemming=True) for doc in raw_docs if doc.strip()]
clean_queries_stemmed = [preprocess_text(query, use_stemming=True) for query in raw_queries if query.strip()]

## # PRZYKŁAD PREPROCESSINGU

In [ ]:
print(f"Tekst oryginalny:\n{raw_docs[0][:200]}...")
print(f"\nTekst po oczyszczeniu (Stemming ON):\n{clean_docs_stemmed[0][:200]}...")

print(f"\nLiczba przetworzonych dokumentów: {len(clean_docs_stemmed)}")
print(f"Liczba przetworzonych zapytań: {len(clean_queries_stemmed)}")

## # BUDOWA MACIERZY I WYSZUKIWANIE DOKUMENTÓW

In [ ]:
def search_and_save(vectorizer, docs_clean, queries_clean, docs_raw, method_name, filename):

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"RAPORT WYNIKÓW WYSZUKIWANIA\n")
        f.write(f"{'='*120}\n")

        print(f"\n{'='*45} METODA: {method_name} {'='*45}")
        
        # 1. tworzenie macierzy
        X_docs = vectorizer.fit_transform(docs_clean)
    
        # 2. transformacja zapytań
        X_queries = vectorizer.transform(queries_clean)
    
        # 3. obliczanie podobieństwa
        similarity_matrix = cosine_similarity(X_queries, X_docs)
    
        # 4. wyświetlanie wyników
        for i, query_scores in enumerate(similarity_matrix):
            top_indices = query_scores.argsort()[-5:][::-1]

            f.write(f"ZAPYTANIE {i+1}: '{raw_queries[i]}'\n")
            f.write(f"-"*120 + "\n")
            
            print(f"\nZapytanie {i+1}: '{raw_queries[i]}'")
            print("-" * 120)
    
            for rank, doc_idx in enumerate(top_indices, 1):
                score = query_scores[doc_idx]
                if score > 0:
                    f.write(f"Rank {rank}. [Dokument {doc_idx+1}] Score: {score:.4f}\n")
                    snippet = docs_raw[doc_idx][:200].replace('\n', ' ')
                    f.write(f"Fragment: {snippet}...\n\n")
                    
                    print(f"[Dokument {doc_idx+1}] Score: {score:.4f}")
                    print(f"  Fragment: {docs_raw[doc_idx][:200].replace(chr(10), ' ')}...")
                else:
                    f.write(f"Rank {rank}. [Brak dopasowania]\n")
                    print(f"[Dokument {doc_idx+1}] Brak dopasowania (Score: 0.0)")

            f.write("\n" + "#"*120 + "\n\n")
            print("-" * 120)

    print(f"Wyniki zapisano do pliku: {filename}")

## # PODGLĄD MACIERZY

In [ ]:
def show_matrix_snippet(vectorizer, docs_clean, title):
    print(f"\n{'='*40} {title} (TOP 10 SŁÓW) {'='*40}")

    # 1. Tworzymy macierz
    X = vectorizer.fit_transform(docs_clean)
    
    feature_names = vectorizer.get_feature_names_out()

    # 2. Sumujemy kolumny
    sum_words = X.sum(axis=0)

    # 3. Tworzymy listę par: (słowo, suma)
    words_freq = [(word, sum_words[0, idx]) for idx, word in enumerate(feature_names)]

    # 4. Sortujemy i bierzemy top 10
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    top_10_words = [x[0] for x in words_freq[:10]]

    # 5. Tworzymy DataFrame
    df_full = pd.DataFrame(X.toarray(), columns=feature_names)

    # 6. Top 10 słów
    df_top = df_full[top_10_words]

    # 7. Wyświetlamy dokumenty
    print(f"Najważniejsze słowa w tej metodzie: {top_10_words}")
    print("Poniżej tabela dla pierwszych 10 dokumentów:")
    display(df_top.head(10))

### ***Binary***

In [ ]:
print("Podgląd macierzy (Wiersze = Dokumenty, Kolumny = Słowa)")

# 1. Binary (Zera i Jedynki)
show_matrix_snippet(CountVectorizer(binary=True), clean_docs_stemmed, "BINARY (0 = brak, 1 = jest)")

### ***Bag Of Words***

In [ ]:
print("Podgląd macierzy (Wiersze = Dokumenty, Kolumny = Słowa)")

# 2. Bag of Words (Liczby całkowite - ile razy słowo wystąpiło)
show_matrix_snippet(CountVectorizer(), clean_docs_stemmed, "BAG OF WORDS (Liczba wystąpień)")

### ***TF-IFD***

In [ ]:
print("Podgląd macierzy (Wiersze = Dokumenty, Kolumny = Słowa)")

# 3. TF-IDF (Ułamki - wagi słów)
show_matrix_snippet(TfidfVectorizer(), clean_docs_stemmed, "TF-IDF (Wagi ważności)")

## # WARIANTY MACIERZY I WYNIKI

### ***Binarny***
Sprawdza tylko czy słowo jest, czy go nie ma (0 lub 1)

In [ ]:
# przy zmianie "use_stemming=True" na "use_stemming=False", trzeba zmienić nazwę pliku, aby nie nadpisać poprzednich - dodaj "_without_stemm"
binary_vectorizer = CountVectorizer(binary=True)
search_and_save(binary_vectorizer, clean_docs_stemmed, clean_queries_stemmed, raw_docs, "Binary (0/1)", "wyniki_BINARY_with_stemm.txt")

#### *BOW (bag of words)*
Zlicza, ile razy słowo wystąpiło 

In [ ]:
# przy zmianie "use_stemming=True" na "use_stemming=False", trzeba zmienić nazwę pliku, aby nie nadpisać poprzednich - dodaj "_without_stemm"
bow_vectorizer = CountVectorizer()
search_and_save(bow_vectorizer, clean_docs_stemmed, clean_queries_stemmed, raw_docs, "Bag of Words (BOW)", "wyniki_BOW_with_stemm.txt")

### *TF-IDF*
Waży słowa: słowa rzadkie (unikalne) są ważniejsze niż częste

In [ ]:
# przy zmianie "use_stemming=True" na "use_stemming=False", trzeba zmienić nazwę pliku, aby nie nadpisać poprzednich - dodaj "_without_stemm"
tfidf_vectorizer = TfidfVectorizer()
search_and_save(tfidf_vectorizer, clean_docs_stemmed, clean_queries_stemmed, raw_docs, "TF-IDF", "wyniki_TFIDF_with_stemm.txt")

## # WIZUALIZACJA (Word Cloud)

In [ ]:
# scalamy dokumenty
all_text = " ".join(clean_docs_stemmed)

# generujemy chmurę słów
wordcloud = WordCloud(width=800, height=400, background_color='white',
                      colormap='cividis', collocations=False).generate(all_text)

In [ ]:
# wyświetlamy wykres
plt.figure(figsize=(18, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("CHMURA SŁÓW DLA ZBIORU DOKUMENTÓW\n (po stemming)\n", 
          fontsize=42, 
          fontweight='bold', 
          color='darkred')
plt.show()

## # ZAPIS CHMURY DO PLIKU

In [ ]:
# przy zmianie "use_stemming=True" na "use_stemming=False", trzeba zmienić nazwę pliku, aby nie nadpisać poprzednich - dodaj "_without_stemm"
wordcloud.to_file("wordcloud_with_stemming.png") 
print("Zapisano chmurę słów do pliku 'wordcloud_with_stemming.png'")

## # LICZBOWE PRZEDSTAWIENIE SŁÓW

In [ ]:
# rozbijamy na pojedyńcze słowa
words_list = all_text.split()

# Counter - zliczamy słowa
word_counts = Counter(words_list)

# ranking
top_10 = word_counts.most_common(10)

print(f"{'Miejsce':<10} {'Słowo':<20} {'Liczba wystąpień'}")
print("-" * 50)
for i, (word, count) in enumerate(top_10, 1):
    print(f"{i:<10} {word:<20} {count}")

# PODSUMOWANIE

## 1. Metoda Binarna (Binary):

- ***Wniosek***: Jest najmniej precyzyjna. Traktuje słowo, które wystąpiło raz, tak samo jak słowo, które wystąpiło 100 razy.

- ***Skutek***: Często w wynikach widzimy wiele dokumentów z taką samą oceną (Score), co utrudnia stworzenie dobrego rankingu.

## 2. Bag of Words (BOW):

- ***Wniosek***: Lepiej niż binarna, bo uwzględnia liczebność.

- ***Wada***: Faworyzuje długie dokumenty. Jeśli dokument jest bardzo długi, ma statystycznie więcej słów kluczowych, przez co może znaleźć się wysoko w rankingu, nawet jeśli słabo pasuje tematycznie.

## 3. TF-IDF (Najlepsza):

- ***Wniosek***: Daje najbardziej trafne wyniki.

- ***Dlaczego***: Mechanizm IDF (Inverse Document Frequency) obniża wagę słów pospolitych, a podbija wagę słów unikalnych dla danego tematu. Dzięki temu, wpisując "ceny ropy", system ignoruje ogólne słowa biznesowe, a skupia się na tych konkretnych artykułach o ropie.